# 0. 라이브러리 설치

In [ ]:
%pip install ipykernal
%pip install librosa
%pip install numpy
%pip install noisereduce
%pip install sklearn
%pip install pickle

In [1]:
import librosa
import numpy as np
import noisereduce as nr
import sklearn.preprocessing
from tqdm import tqdm
from sklearn.svm import SVC
import pickle

c:\Users\GAILAB\anaconda3\envs\fmcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 데이터 전처리

In [2]:
def zero_padding(mfcc, max_length=40):
    if mfcc.shape[1] < max_length:
        padding = np.zeros((mfcc.shape[0], max_length - mfcc.shape[1]))
        mfcc = np.hstack((mfcc, padding))
    else:
        mfcc = mfcc[:, :max_length]
        
    mfcc = sklearn.preprocessing.minmax_scale(mfcc, axis=1)
    return mfcc


def raw_to_mfcc(raw_file_path, sampling_rate=16000):
    # read raw file
    with open(raw_file_path, 'rb') as file:
        audio_data = np.frombuffer(file.read(), dtype=np.int16)
        audio_data = audio_data.astype(np.float32)

    # reduce noise
    wav_audio = nr.reduce_noise(y=audio_data, sr=sampling_rate)
    
    # extract mfcc
    mfcc = librosa.feature.mfcc(y=wav_audio, sr=sampling_rate, n_mfcc=13, n_mels=100, fmin=0, fmax=400)
    
    # zero padding
    mfcc = zero_padding(mfcc)

    return mfcc

In [3]:
X_test = []
X_test_name = []

with open('fmcc_test.ctl', 'r') as file:
    for line in tqdm(file):
        file_path = './raw16k/test/' + line.rstrip() + '.raw'
        
        X_test_name.append(file_path)
        X_test.append(raw_to_mfcc(file_path))

X_test = [data.flatten() for data in X_test]

1000it [00:34, 28.58it/s]


# 2. 학습된 모델 로드

In [4]:
svm_classifier = SVC()

with open('svm_classifier.pkl', 'rb') as f:
    svm_classifier = pickle.load(f)

# 3. 테스트 데이터 추론 & 결과 저장

In [5]:
# Predict the classes for the preprocessed_data
predict = svm_classifier.predict(X_test)

In [6]:
file = open('./과탑_test_results.txt', 'w')

for i in range(len(predict)):
    if predict[i] == 'M':
        res = 'male'
    else:
        res = 'feml'

    file.write(X_test_name[i] + ' ' + res + '\n')
file.close()